In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense


In [2]:
merged_df_full = pd.read_csv('merged_df_full.csv')

# Display the first few rows of the DataFrame
print(merged_df_full.head())



         Date  Tair_f_avg      evap     GWS_tavg  Rain_f_ta   SWE_tavg  \
0  1965-01-02  260.363312  0.000074  1355.867798   0.000001  22.969460   
1  1965-01-03  263.606293  0.000081  1355.743408   0.000020  18.499897   
2  1965-01-04  266.839752  0.000080  1355.505737   0.000092  17.172014   
3  1965-01-05  263.835114  0.000034  1355.300537   0.000028  16.777233   
4  1965-01-06  261.595673  0.000060  1355.049561   0.000000  14.420487   

          Q  value_mgL  
0  0.003078        NaN  
1  0.002452        NaN  
2  0.002017        NaN  
3  0.001921        NaN  
4  0.001872        NaN  


In [3]:
import pandas as pd

# Assuming merged_df_full is your merged dataframe

# Convert 'Date' column to datetime if it's not already
merged_df_full['Date'] = pd.to_datetime(merged_df_full['Date'])

# Sort the dataframe by 'Date'
merged_df_full = merged_df_full.sort_values(by='Date')

# Extract unique years from the 'Date' column
years = merged_df_full['Date'].dt.year.unique()

# Split the years into groups of 4 (3 years in df1 and 1 year in df2)
groups = [years[i:i+4] for i in range(0, len(years), 4)]

# Create empty lists to store datasets
df1_list = []
df2_list = []

# Iterate over groups
for i, group in enumerate(groups):
    # Extract data for df1 (first 3 years)
    df1_data = merged_df_full[merged_df_full['Date'].dt.year.isin(group[:-1])]
    df1_list.append(df1_data)
    
    # Extract data for df2 (last year in group)
    df2_data = merged_df_full[merged_df_full['Date'].dt.year == group[-1]]
    df2_list.append(df2_data)

# Concatenate dataframes for df1 and df2
df1 = pd.concat(df1_list)
df2 = pd.concat(df2_list)

# Reset index
df1.reset_index(drop=True, inplace=True)
df2.reset_index(drop=True, inplace=True)

# Print resulting dataframes
print("Dataset df1:")
print(df1)
print("\nDataset df2:")
print(df2)



Dataset df1:
           Date  Tair_f_avg      evap     GWS_tavg     Rain_f_ta   SWE_tavg  \
0    1965-01-02  260.363312  0.000074  1355.867798  1.472223e-06  22.969460   
1    1965-01-03  263.606293  0.000081  1355.743408  2.042398e-05  18.499897   
2    1965-01-04  266.839752  0.000080  1355.505737  9.216375e-05  17.172014   
3    1965-01-05  263.835114  0.000034  1355.300537  2.777230e-05  16.777233   
4    1965-01-06  261.595673  0.000060  1355.049561  0.000000e+00  14.420487   
...         ...         ...       ...          ...           ...        ...   
3277 1975-12-27  267.970428  0.000016  1638.014404  4.263832e-06   3.854585   
3278 1975-12-28  265.627380  0.000018  1637.338745  4.013156e-05   5.077522   
3279 1975-12-29  258.809601  0.000013  1636.739380  4.151054e-08   6.038525   
3280 1975-12-30  264.983215  0.000012  1636.322998  1.332024e-04   9.480419   
3281 1975-12-31  266.911194  0.000028  1637.509766  2.616605e-04  17.256781   

             Q  value_mgL  
0     0.00

In [4]:
# Create X_train from df1 excluding 'value_mgL' column
X_train = df1.drop(columns=['value_mgL'])
y_train = df1["value_mgL"]
X_train = X_train.drop(columns=['Date'])

# Drop 'Date' column from X_test

# Print X_train
print("X_train:")
print(X_train)

# Print y_train
print("y_train:")
print(y_train)

X_train:
      Tair_f_avg      evap     GWS_tavg     Rain_f_ta   SWE_tavg         Q
0     260.363312  0.000074  1355.867798  1.472223e-06  22.969460  0.003078
1     263.606293  0.000081  1355.743408  2.042398e-05  18.499897  0.002452
2     266.839752  0.000080  1355.505737  9.216375e-05  17.172014  0.002017
3     263.835114  0.000034  1355.300537  2.777230e-05  16.777233  0.001921
4     261.595673  0.000060  1355.049561  0.000000e+00  14.420487  0.001872
...          ...       ...          ...           ...        ...       ...
3277  267.970428  0.000016  1638.014404  4.263832e-06   3.854585  0.004126
3278  265.627380  0.000018  1637.338745  4.013156e-05   5.077522  0.002840
3279  258.809601  0.000013  1636.739380  4.151054e-08   6.038525  0.002349
3280  264.983215  0.000012  1636.322998  1.332024e-04   9.480419  0.002096
3281  266.911194  0.000028  1637.509766  2.616605e-04  17.256781  0.001945

[3282 rows x 6 columns]
y_train:
0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
  

In [5]:
# Create X_train from df1 excluding 'value_mgL' column
X_test = df2.drop(columns=['value_mgL'])
y_test = df2["value_mgL"]
X_test = X_test.drop(columns=['Date'])
# Print X_train
print("X_test:")
print(X_test)

# Print y_train
print("y_test:")
print(y_test)

X_test:
      Tair_f_avg      evap     GWS_tavg  Rain_f_ta   SWE_tavg         Q
0     256.108734  0.000035  1459.538452   0.000041  15.254924  0.002129
1     250.902328  0.000017  1459.268555   0.000000  15.079142  0.001957
2     260.273529  0.000022  1459.030762   0.000000  13.692603  0.001896
3     261.424988  0.000017  1458.771606   0.000000  12.156770  0.001849
4     264.160095  0.000018  1458.487183   0.000000  11.247462  0.001735
...          ...       ...          ...        ...        ...       ...
1457  262.728699  0.000030  1681.091675   0.000000  20.298054  0.005592
1458  260.893188  0.000052  1680.958740   0.000139  23.501686  0.003784
1459  264.405060  0.000044  1680.829468   0.000029  26.433260  0.003059
1460  261.499847  0.000050  1680.640381   0.000000  23.369114  0.002903
1461  263.117340  0.000027  1680.375244   0.000000  20.688452  0.002670

[1462 rows x 6 columns]
y_test:
0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
1457   NaN
1458   NaN
1459   

In [6]:
print("Shape of X_train:", X_train.shape)


Shape of X_train: (3282, 6)


In [7]:

# Custom loss function
def custom_loss(y_true, y_pred):
    # Mask for non-NaN values in y_true
    mask = tf.cast(~tf.math.is_nan(y_true), dtype=tf.float32)
    
    # Compute mean squared error only for non-NaN values
    loss = tf.keras.losses.mean_squared_error(y_true, y_pred)
    loss = loss * mask
    
    # Compute mean of the loss only for non-NaN values
    loss = tf.reduce_sum(loss) / tf.reduce_sum(mask)
    
    return loss


# Convert DataFrame to NumPy array
X_train_np = X_train.values

# Reshape X_train_np to have shape (3282, 1, 6)
X_train_np = X_train_np.reshape(X_train_np.shape[0], 1, X_train_np.shape[1])

# Define LSTM model with custom loss
def create_lstm_model(input_shape):
    model = Sequential([
        LSTM(64, input_shape=input_shape),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss=custom_loss)
    return model

# Create and train LSTM model
model = create_lstm_model(input_shape=(1, X_train_np.shape[2]))
# model.fit(X_train_np, y_train, epochs=5000, batch_size=32)

In [8]:
y_train

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
3277   NaN
3278   NaN
3279   NaN
3280   NaN
3281   NaN
Name: value_mgL, Length: 3282, dtype: float64

In [9]:
non_nan_count_train = np.sum(~np.isnan(y_train))

# Calculate non-NaN values in y_test
non_nan_count_test = np.sum(~np.isnan(y_test))

print("Number of non-NaN values in y_train:", non_nan_count_train)
print("Number of non-NaN values in y_test:", non_nan_count_test)

Number of non-NaN values in y_train: 92
Number of non-NaN values in y_test: 42


In [10]:
import numpy as np

# Assuming y_train is your array
y_train = np.nan_to_num(y_train, nan=0.0)
y_test = np.nan_to_num(y_test, nan=0.0)

In [11]:
non_nan_count_train = np.sum(np.isnan(y_train))

# Calculate non-NaN values in y_test
non_nan_count_test = np.sum(np.isnan(y_test))

print("Number of non-NaN values in y_train:", non_nan_count_train)
print("Number of non-NaN values in y_test:", non_nan_count_test)


Number of non-NaN values in y_train: 0
Number of non-NaN values in y_test: 0


In [12]:

# Convert DataFrame to NumPy array
X_train_np = X_train.values

# Reshape X_train_np to have shape (3282, 1, 6)
X_train_np = X_train_np.reshape(X_train_np.shape[0], 1, X_train_np.shape[1])

# Define LSTM model with custom loss
# Function to create LSTM model


In [13]:


# Custom loss function
def custom_loss(y_true, y_pred):
    # Mask for non-NaN values in y_true
    mask = tf.cast(~tf.math.is_nan(y_true), dtype=tf.float32)
    
    # Compute mean squared error only for non-NaN values
    loss = tf.keras.losses.mean_squared_error(y_true, y_pred)
    loss = loss * mask
    
    # Compute mean of the loss only for non-NaN values
    loss = tf.reduce_sum(loss) / tf.reduce_sum(mask)
    
    return loss

In [14]:
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    for device in physical_devices:
        print(f"Device name: {device.name}")
else:
    print("No GPU devices available.")

    

No GPU devices available.


In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from sklearn.base import BaseEstimator
import numpy as np

# Custom loss function
def custom_loss(y_true, y_pred):
    mask = tf.cast(~tf.math.is_nan(y_true), dtype=tf.float32)
    loss = tf.keras.losses.mean_squared_error(y_true, y_pred)
    loss = loss * mask
    loss = tf.reduce_sum(loss) / tf.reduce_sum(mask)
    return loss

# Keras model class implementing fit method
class KerasLSTM(BaseEstimator):
    def __init__(self, neurons=64, optimizer='adam', learning_rate=0.001, momentum=0.9, activation='relu', epochs=100, batch_size=32):
        self.neurons = neurons
        self.optimizer = optimizer
        self.learning_rate = learning_rate
        self.momentum = momentum
        self.activation = activation
        self.epochs = epochs
        self.batch_size = batch_size
    
    def fit(self, X, y):
        opt = None
        if self.optimizer == 'adam':
            opt = Adam(learning_rate=self.learning_rate)
        elif self.optimizer == 'rmsprop':
            opt = RMSprop(learning_rate=self.learning_rate)
        elif self.optimizer == 'sgd':
            opt = SGD(learning_rate=self.learning_rate, momentum=self.momentum)
        
        with tf.device('/GPU:0'):  # Change 1: Specify GPU device
            model = Sequential([
                LSTM(self.neurons, input_shape=(1, X.shape[2]), return_sequences=True),
                Dense(1, activation=self.activation)
            ])
            model.compile(optimizer=opt, loss=custom_loss)
            self.model = model
        
            self.model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size, verbose=1)  # Change 2: Set verbose=1
    
    def predict(self, X):
        return self.model.predict(X)
    
    def score(self, X, y):
        # Placeholder score method
        return np.mean(np.abs(self.predict(X) - y))

# Define parameter grid for GridSearchCV
param_grid = {
    'neurons': [32, 64, 128],
    'optimizer': ['adam', 'rmsprop', 'sgd'],
    'learning_rate': [0.001, 0.01],
    'momentum': [0.9, 0.95],
    'activation': ['relu', 'tanh']
}

# Instantiate GridSearchCV
grid_search = GridSearchCV(estimator=KerasLSTM(), param_grid=param_grid, cv=3)

# Fit GridSearchCV to the data
grid_search.fit(X_train_np, y_train)

# Retrieve best hyperparameters and best model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_


Epoch 1/100

69/69 [==============================] - 1s 1ms/step - loss: 6.7331
Epoch 2/100
69/69 [==============================] - 0s 1ms/step - loss: 6.6565
Epoch 3/100
69/69 [==============================] - 0s 1ms/step - loss: 6.6565
Epoch 4/100
69/69 [==============================] - 0s 1ms/step - loss: 6.5668
Epoch 5/100
69/69 [==============================] - 0s 1ms/step - loss: 6.6565
Epoch 6/100
69/69 [==============================] - 0s 1ms/step - loss: 6.6565
Epoch 7/100
69/69 [==============================] - 0s 2ms/step - loss: 6.6565
Epoch 8/100
69/69 [==============================] - 0s 1ms/step - loss: 6.5969
Epoch 9/100
69/69 [==============================] - 0s 1ms/step - loss: 6.5532
Epoch 10/100
69/69 [==============================] - 0s 1ms/step - loss: 6.6565
Epoch 11/100
69/69 [==============================] - 0s 1ms/step - loss: 6.6565
Epoch 12/100
69/69 [==============================] - 0s 1ms/step - loss: 6.6565
Epoch 13/100
69/69 [================

69/69 [==============================] - 0s 1ms/step - loss: 6.5982
Epoch 100/100
35/35 [==============================] - 0s 1ms/step
Epoch 1/100
69/69 [==============================] - 1s 2ms/step - loss: 7.3044
Epoch 2/100
69/69 [==============================] - 0s 1ms/step - loss: 6.4683
Epoch 3/100
69/69 [==============================] - 0s 2ms/step - loss: 6.5355
Epoch 4/100
69/69 [==============================] - 0s 2ms/step - loss: 6.5305
Epoch 5/100
69/69 [==============================] - 0s 2ms/step - loss: 6.5391
Epoch 6/100
69/69 [==============================] - 0s 2ms/step - loss: 6.5323
Epoch 7/100
69/69 [==============================] - 0s 1ms/step - loss: 6.5361
Epoch 8/100
69/69 [==============================] - 0s 2ms/step - loss: 6.5373
Epoch 9/100
69/69 [==============================] - 0s 2ms/step - loss: 6.5363
Epoch 10/100
69/69 [==============================] - 0s 2ms/step - loss: 6.4126
Epoch 11/100
69/69 [==============================] - 0s 2ms/ste

69/69 [==============================] - 0s 2ms/step - loss: 6.3805
Epoch 100/100
35/35 [==============================] - 0s 963us/step
Epoch 1/100
69/69 [==============================] - 2s 2ms/step - loss: 6.1076
Epoch 2/100
69/69 [==============================] - 0s 2ms/step - loss: 5.9305
Epoch 3/100
69/69 [==============================] - 0s 2ms/step - loss: 6.0038
Epoch 4/100
69/69 [==============================] - 0s 1ms/step - loss: 6.0152
Epoch 5/100
69/69 [==============================] - 0s 2ms/step - loss: 6.0282
Epoch 6/100
69/69 [==============================] - 0s 2ms/step - loss: 6.0073
Epoch 7/100
69/69 [==============================] - 0s 1ms/step - loss: 5.9721
Epoch 8/100
69/69 [==============================] - 0s 1ms/step - loss: 5.9647
Epoch 9/100
69/69 [==============================] - 0s 1ms/step - loss: 6.0228
Epoch 10/100
69/69 [==============================] - 0s 1ms/step - loss: 6.0043
Epoch 11/100
69/69 [==============================] - 0s 1ms/s

69/69 [==============================] - 0s 1ms/step - loss: 6.0117
Epoch 100/100
35/35 [==============================] - 0s 1ms/step
Epoch 1/100
69/69 [==============================] - 1s 2ms/step - loss: 6.6565
Epoch 2/100
69/69 [==============================] - 0s 1ms/step - loss: 6.6565
Epoch 3/100
69/69 [==============================] - 0s 1ms/step - loss: 6.6047
Epoch 4/100
69/69 [==============================] - 0s 1ms/step - loss: 6.6565
Epoch 5/100
69/69 [==============================] - 0s 1ms/step - loss: 6.6565
Epoch 6/100
69/69 [==============================] - 0s 1ms/step - loss: 6.5893
Epoch 7/100
69/69 [==============================] - 0s 1ms/step - loss: 6.6565
Epoch 8/100
69/69 [==============================] - 0s 1ms/step - loss: 6.6565
Epoch 9/100
69/69 [==============================] - 0s 1ms/step - loss: 6.6565
Epoch 10/100
69/69 [==============================] - 0s 1ms/step - loss: 6.6565
Epoch 11/100
69/69 [==============================] - 0s 2ms/ste

69/69 [==============================] - 0s 1ms/step - loss: 6.6565
Epoch 100/100
35/35 [==============================] - 0s 849us/step
Epoch 1/100
69/69 [==============================] - 1s 2ms/step - loss: 6.5171
Epoch 2/100
69/69 [==============================] - 0s 2ms/step - loss: 6.5409
Epoch 3/100
69/69 [==============================] - 0s 1ms/step - loss: 6.5417
Epoch 4/100
69/69 [==============================] - 0s 1ms/step - loss: 6.5424
Epoch 5/100
69/69 [==============================] - 0s 1ms/step - loss: 6.5402
Epoch 6/100
69/69 [==============================] - 0s 1ms/step - loss: 6.5435
Epoch 7/100
69/69 [==============================] - 0s 1ms/step - loss: 6.5394
Epoch 8/100
69/69 [==============================] - 0s 1ms/step - loss: 6.4913
Epoch 9/100
69/69 [==============================] - 0s 1ms/step - loss: 6.5419
Epoch 10/100
69/69 [==============================] - 0s 1ms/step - loss: 6.4798
Epoch 11/100
69/69 [==============================] - 0s 1ms/s

...

69/69 [==============================] - 0s 2ms/step - loss: 6.6737
Epoch 82/100
69/69 [==============================] - 0s 2ms/step - loss: 6.6253
Epoch 83/100
69/69 [==============================] - 0s 2ms/step - loss: 6.7287
Epoch 84/100
69/69 [==============================] - 0s 1ms/step - loss: 6.8332
Epoch 85/100
69/69 [==============================] - 0s 1ms/step - loss: 6.6861
Epoch 86/100
69/69 [==============================] - 0s 1ms/step - loss: 6.7666
Epoch 87/100
69/69 [==============================] - 0s 1ms/step - loss: 6.5648
Epoch 88/100
69/69 [==============================] - 0s 2ms/step - loss: 6.6869
Epoch 89/100
69/69 [==============================] - 0s 1ms/step - loss: 6.7219
Epoch 90/100
69/69 [==============================] - 0s 2ms/step - loss: 6.5911
Epoch 91/100
69/69 [==============================] - 0s 2ms/step - loss: 6.6693
Epoch 92/100
69/69 [==============================] - 0s 1ms/step - loss: 6.7309
Epoch 93/100
69/69 [=====================

69/69 [==============================] - 0s 1ms/step - loss: 33.9152
Epoch 81/100
69/69 [==============================] - 0s 1ms/step - loss: 33.9152
Epoch 82/100
69/69 [==============================] - 0s 1ms/step - loss: 33.8903
Epoch 83/100
69/69 [==============================] - 0s 1ms/step - loss: 33.9152
Epoch 84/100
69/69 [==============================] - 0s 1ms/step - loss: 33.9152
Epoch 85/100
69/69 [==============================] - 0s 1ms/step - loss: 33.9152
Epoch 86/100
69/69 [==============================] - 0s 1ms/step - loss: 33.8888
Epoch 87/100
69/69 [==============================] - 0s 2ms/step - loss: 33.9152
Epoch 88/100
69/69 [==============================] - 0s 1ms/step - loss: 33.9152
Epoch 89/100
69/69 [==============================] - 0s 1ms/step - loss: 33.9089
Epoch 90/100
69/69 [==============================] - 0s 1ms/step - loss: 33.8911
Epoch 91/100
69/69 [==============================] - 0s 1ms/step - loss: 33.8627
Epoch 92/100
69/69 [=========

69/69 [==============================] - 0s 1ms/step - loss: 43.6811
Epoch 80/100
69/69 [==============================] - 0s 1ms/step - loss: 43.6811
Epoch 81/100
69/69 [==============================] - 0s 2ms/step - loss: 43.6811
Epoch 82/100
69/69 [==============================] - 0s 2ms/step - loss: 43.6811
Epoch 83/100
69/69 [==============================] - 0s 1ms/step - loss: 43.6811
Epoch 84/100
69/69 [==============================] - 0s 1ms/step - loss: 43.6811
Epoch 85/100
69/69 [==============================] - 0s 1ms/step - loss: 43.4444
Epoch 86/100
69/69 [==============================] - 0s 1ms/step - loss: 43.5395
Epoch 87/100
69/69 [==============================] - 0s 1ms/step - loss: 43.5859
Epoch 88/100
69/69 [==============================] - 0s 1ms/step - loss: 43.6811
Epoch 89/100
69/69 [==============================] - 0s 1ms/step - loss: 43.6811
Epoch 90/100
69/69 [==============================] - 0s 1ms/step - loss: 43.5507
Epoch 91/100
69/69 [=========

69/69 [==============================] - 0s 1ms/step - loss: 33.4183
Epoch 79/100
69/69 [==============================] - 0s 2ms/step - loss: 33.4454
Epoch 80/100
69/69 [==============================] - 0s 2ms/step - loss: 33.4454
Epoch 81/100
69/69 [==============================] - 0s 2ms/step - loss: 33.4454
Epoch 82/100
69/69 [==============================] - 0s 2ms/step - loss: 33.4454
Epoch 83/100
69/69 [==============================] - 0s 2ms/step - loss: 33.4454
Epoch 84/100
69/69 [==============================] - 0s 2ms/step - loss: 33.4266
Epoch 85/100
69/69 [==============================] - 0s 2ms/step - loss: 33.4454
Epoch 86/100
69/69 [==============================] - 0s 2ms/step - loss: 33.4454
Epoch 87/100
69/69 [==============================] - 0s 2ms/step - loss: 33.4454
Epoch 88/100
69/69 [==============================] - 0s 2ms/step - loss: 33.4454
Epoch 89/100
69/69 [==============================] - 0s 2ms/step - loss: 33.4454
Epoch 90/100
69/69 [=========

103/103 [==============================] - 0s 2ms/step - loss: 33.6661
Epoch 76/100
103/103 [==============================] - 0s 2ms/step - loss: 33.6725
Epoch 77/100
103/103 [==============================] - 0s 2ms/step - loss: 33.6661
Epoch 78/100
103/103 [==============================] - 0s 2ms/step - loss: 33.6530
Epoch 79/100
103/103 [==============================] - 0s 2ms/step - loss: 33.6725
Epoch 80/100
103/103 [==============================] - 0s 2ms/step - loss: 33.6634
Epoch 81/100
103/103 [==============================] - 0s 2ms/step - loss: 33.6725
Epoch 82/100
103/103 [==============================] - 0s 2ms/step - loss: 33.6603
Epoch 83/100
103/103 [==============================] - 0s 2ms/step - loss: 33.6594
Epoch 84/100
103/103 [==============================] - 0s 2ms/step - loss: 33.6725
Epoch 85/100
103/103 [==============================] - 0s 2ms/step - loss: 33.6725
Epoch 86/100
103/103 [==============================] - 0s 2ms/step - loss: 33.6725
Epoch

In [22]:
print(best_model)

KerasLSTM(activation='tanh', learning_rate=0.01, optimizer='sgd')


In [16]:
# import tensorflow as tf

# # Custom loss function
# def custom_loss1(y_true, y_pred):
#     # Mask for non-zero values in y_true
#     mask = tf.cast(y_true != 0, dtype=tf.float32)
    
#     # Compute mean squared error only for non-zero values
#     loss = tf.keras.losses.mean_squared_error(y_true, y_pred)
#     loss = loss * mask
    
#     # Compute mean of the loss only for non-zero values
#     loss = tf.reduce_sum(loss) / tf.reduce_sum(mask)
    
#     return loss


# # Convert DataFrame to NumPy array
# X_train_np = X_train.values

# # Reshape X_train_np to have shape (3282, 1, 6)
# X_train_np = X_train_np.reshape(X_train_np.shape[0], 1, X_train_np.shape[1])

# # Define LSTM model with custom loss
# def create_lstm_model(input_shape):
#     model1 = Sequential([
#         LSTM(64, input_shape=input_shape),
#         Dense(1)
#     ])
#     model1.compile(optimizer='adam', loss=custom_loss1)
#     return model

# # Create and train LSTM model 
# model1 = create_lstm_model(input_shape=(1, X_train_np.shape[2]))
# model1.fit(X_train_np, y_train, epochs=500, batch_size=32)

In [17]:
# Count non-zero values in y_train
non_zero_count = np.count_nonzero(y_train)

print("Count of non-zero values in y_train:", non_zero_count)

# Print non-zero values in y_train
non_zero_values = y_train[y_train != 0]
print("Non-zero values in y_train:")
print(non_zero_values)


Count of non-zero values in y_train: 92
Non-zero values in y_train:
[1.96792722 1.95390398 1.93988075 1.77160194 1.95390398 2.33720572
 2.21567102 2.2857872  2.21099661 1.89313663 1.93988075 2.0427178
 2.01934574 2.06141545 1.84639252 1.77160194 1.97260163 2.14555485
 2.72518187 3.18327419 3.04771626 3.08978596 3.0617395  2.67376335
 2.55222865 2.58027512 2.27643837 2.03336898 2.43536836 2.91683274
 2.9822745  2.8981351  2.95422803 2.9822745  2.75790275 2.66441452
 2.92150715 2.93085598 2.52418218 1.75290429 2.29981043 2.97292568
 3.03369303 3.21599507 3.40297153 3.11315802 2.96825127 2.8981351
 2.71115864 2.74855393 2.25774073 2.69246099 3.31415771 3.09913479
 2.99162332 3.23936713 3.08978596 2.33720572 2.3278569  3.39362271
 3.21132066 2.75790275 2.92618156 2.51015895 2.40732189 2.85606539
 2.48678689 2.59897276 3.31883213 2.52418218 2.57560071 2.77192599
 2.37927543 2.14555485 2.38394984 2.63169364 3.1458789  3.2674136
 3.36090183 3.34220418 3.01966979 2.90748392 3.01032097 2.874763

In [25]:
y_pred = best_model.predict(X_train_np)

# Print y_pred
print("Predicted values (y_pred):")
print(y_pred)

103/103 [==============================] - 0s 865us/step
Predicted values (y_pred):
[[[1.]]

 [[1.]]

 [[1.]]

 ...

 [[1.]]

 [[1.]]

 [[1.]]]


In [19]:
y_pred

array([[ 0.19490364],
       [ 0.18686566],
       [ 0.18863691],
       ...,
       [-0.07213974],
       [-0.06538811],
       [-0.03495118]], dtype=float32)

In [26]:
import numpy as np

# Count unique values in y_pred
unique_values = np.unique(y_pred)
count_unique_values = len(unique_values)

print("Count of unique values in y_pred:", count_unique_values)


Count of unique values in y_pred: 1
